# 실습 세션 2: AutoGen 기반 여행 플래너 Multi-Agent 앱 구현

이 세션에서는 Microsoft AutoGen 프레임워크를 활용하여 여행 플래너 멀티 에이전트 애플리케이션을 구현합니다. 다양한 역할의 AI 에이전트가 협업하여 사용자의 여행 계획을 도와주는 시스템을 만들어봅니다.

## 1. 필요한 라이브러리 설치

AutoGen 및 관련 라이브러리를 설치합니다.

In [64]:
# AutoGen 및 필요한 라이브러리 설치
!pip install -r requirements.txt

## 2. AutoGen 프레임워크 및 환경 설정

OpenAI API 키 등 환경 변수를 설정하고, 모델 클라이언트를 준비합니다.

In [70]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), # For key-based authentication.
)

print("AutoGen 설정이 완료되었습니다.")

AutoGen 설정이 완료되었습니다.


## 3. 여행 플래너 Multi-Agent 시스템 구성

여행 플래너, 로컬 추천, 언어 팁, 여행 요약 등 다양한 역할의 에이전트를 정의합니다.

In [71]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio

planner_agent = AssistantAgent(
    name="planner_agent",
    description="A helpful assistant that can plan trips",
    model_client=model_client,
    system_message="""You are a helpful assistant that can suggest a travel plan for a user based on their request.\nEnd your planning with 'INITIAL PLAN COMPLETE'."""
)

local_agent = AssistantAgent(
    name="local_agent",
    description="A local assistant that can suggest local activities or places to visit",
    model_client=model_client,
    system_message="""You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user.\nEnd your recommendations with 'LOCAL RECOMMENDATIONS COMPLETE'."""
)

language_agent = AssistantAgent(
    name="language_agent",
    description="A helpful assistant that can provide language tips for a given destination",
    model_client=model_client,
    system_message="""You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination.\nEnd your language tips with 'LANGUAGE TIPS COMPLETE'."""
)

travel_summary_agent = AssistantAgent(
    name="travel_summary_agent",
    description="A helpful assistant that can summarize the travel plan",
    model_client=model_client,
    system_message="""You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan.\nYOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN.\nWhen the plan is complete and all perspectives are integrated, you can respond with TERMINATE."""
)

user_proxy_travel = UserProxyAgent(
    name="user_proxy",
    description="A traveler seeking travel planning assistance"
)

## 4. 그룹 채팅 및 종료 조건 설정

SelectorGroupChat을 사용하여 에이전트 협업 구조를 만들고, 대화 종료 조건과 역할 선택 프롬프트를 정의합니다.

In [72]:
termination = MaxMessageTermination(max_messages=30) | TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent, user_proxy_travel],
    termination_condition=termination
)

## 5. 여행 플래너 Multi-Agent 대화 실행

아래 코드를 실행하면, 다양한 에이전트가 협업하여 여행 계획을 완성합니다.

In [73]:
travel_task = """I'm planning a 5-day trip to Paris with my friend. We love art, food, and want to experience both famous sights and some local hidden gems. We don't speak French. Please help create a detailed travel plan."""

async def run_travel_planner():
    await Console(group_chat.run_stream(task=travel_task))

# 실행:
asyncio.run(run_travel_planner())

---------- TextMessage (user) ----------
I'm planning a 5-day trip to Paris with my friend. We love art, food, and want to experience both famous sights and some local hidden gems. We don't speak French. Please help create a detailed travel plan.
I'm planning a 5-day trip to Paris with my friend. We love art, food, and want to experience both famous sights and some local hidden gems. We don't speak French. Please help create a detailed travel plan.
---------- TextMessage (planner_agent) ----------
Here's a detailed 5-day travel plan for you and your friend to experience the best of Paris:

### Day 1: Arrival and Montmartre Exploration
- **Morning:** Arrive in Paris, check into your accommodation. Consider staying in the Montmartre area for an artistic vibe.
- **Lunch:** Enjoy a casual meal at Le Consulat, a charming café in Montmartre.
- **Afternoon:**
  - Explore the streets of Montmartre: visit the iconic Sacré-Cœur Basilica for breathtaking views of the city.
  - Stroll through Plac

## 6. 마무리 및 다음 단계

이 세션에서는 AutoGen을 활용한 여행 플래너 Multi-Agent 시스템을 구현했습니다. 다양한 역할의 에이전트가 협업하여 사용자의 요구에 맞는 여행 일정을 자동으로 생성하는 과정을 실습했습니다.

- 다양한 역할의 에이전트 설계 및 협업 구조 이해
- SelectorGroupChat을 통한 동적 역할 분배
- 실전 여행 플래너 예제 구현

더 많은 예제와 심화 내용은 [AutoGen 공식 문서](https://microsoft.github.io/autogen/)를 참고하세요.